<a href="https://colab.research.google.com/github/FabiusPaulus/Projekt24/blob/main/Nierenanalyse_nur_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Biblotheken importieren

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')
%matplotlib inline
pd.set_option('display.max_columns', 26)

In [ ]:
#Einlesen der Daten
df= pd.read_csv('/chronic_kidney_disease.csv')
df.head()

In [ ]:
#Spalte ID entfernen
df.drop('id', axis = 1, inplace = True)

In [ ]:
#Spaltennamen ändern

df.columns = ['age', 'blood_pressure', 'specific_gravity', 'albumin', 'sugar', 'red_blood_cells', 'pus_cell',
              'pus_cell_clumps', 'bacteria', 'blood_glucose_random', 'blood_urea', 'serum_creatinine', 'sodium',
              'potassium', 'haemoglobin', 'packed_cell_volume', 'white_blood_cell_count', 'red_blood_cell_count',
              'hypertension', 'diabetes_mellitus', 'coronary_artery_disease', 'appetite', 'peda_edema',
              'aanemia', 'class']

In [ ]:
#Vorschau der Daten
df.head()

# **Explorative Datenanalyse**


In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
# Konvertierung Datentyp

df['packed_cell_volume'] = pd.to_numeric(df['packed_cell_volume'], errors='coerce')
df['white_blood_cell_count'] = pd.to_numeric(df['white_blood_cell_count'], errors='coerce')
df['red_blood_cell_count'] = pd.to_numeric(df['red_blood_cell_count'], errors='coerce')

In [ ]:
# Datentypen zeigen
df.info()

In [ ]:
#Extrahierung von nummerischen und kategorischen Spalten

cat_cols = [col for col in df.columns if df[col].dtype == 'object']
num_cols = [col for col in df.columns if df[col].dtype != 'object']

In [ ]:
#Ausgabe kategorischer Spalten
print(cat_cols)

In [ ]:
#Darstellung der möglichen Werte in kategorischen Spalten

for col in cat_cols:
    print(f"{col} has {df[col].unique()} values\n")

In [ ]:
#Ersetzen von fehlerhaften Einträgen

df['diabetes_mellitus'].replace(to_replace = {'\tno':'no','\tyes':'yes',' yes':'yes'},inplace=True)

df['coronary_artery_disease'] = df['coronary_artery_disease'].replace(to_replace = '\tno', value='no')

df['class'] = df['class'].replace(to_replace = {'ckd\t': 'ckd', 'notckd': 'not ckd'})

In [ ]:
#Alle 0 und 1 zu int oder float konvertieren oder als NaN deklarieren

df['class'] = df['class'].map({'ckd': 0, 'not ckd': 1})
df['class'] = pd.to_numeric(df['class'], errors='coerce')

In [ ]:
# auf Normalverteilung prüfen

plt.figure(figsize = (20, 15))
plotnumber = 1

for column in num_cols:
    if plotnumber <= 14:
        ax = plt.subplot(3, 5, plotnumber)
        sns.distplot(df[column])
        plt.xlabel(column)

    plotnumber += 1

plt.tight_layout()
plt.show()

In [ ]:
# kategorische Spalten betrachten

plt.figure(figsize = (20, 15))
plotnumber = 1

for column in cat_cols:
    if plotnumber <= 11:
        ax = plt.subplot(3, 4, plotnumber)
        sns.countplot(df[column], palette = 'rocket')
        plt.xlabel(column)

    plotnumber += 1

plt.tight_layout()
plt.show()

In [ ]:
# heatmap der Daten erstellen

plt.figure(figsize = (15, 8))

sns.heatmap(df.corr(), annot = True, linewidths = 2, linecolor = 'lightgrey')
plt.show()

In [ ]:
def kde(col):
    grid = sns.FacetGrid(df, hue="class", height = 6, aspect=2)
    grid.map(sns.kdeplot, col)
    grid.add_legend()

In [ ]:
  def kde(col):
    plt.figure(figsize=(8, 6))
    # Replace "?" with NaN and then convert to numeric
    df_numeric_ckd = pd.to_numeric(df[df['class'] == 0][col].replace('?', np.nan), errors='coerce')
    sns.kdeplot(df_numeric_ckd, shade=True, label='CKD', color='blue')  # Set color for CKD class
    # Replace "?" with NaN and then convert to numeric
    df_numeric_notckd = pd.to_numeric(df[df['class'] == 1][col].replace('?', np.nan), errors='coerce')
    sns.kdeplot(df_numeric_notckd, shade=True, label='Not CKD', color='orange')  # Set color for non-CKD class
    plt.xlabel(col)
    plt.title(f'KDE plot for {col} with class distribution')
    plt.legend()
    plt.show()

kde('red_blood_cell_count')
kde('packed_cell_volume')
kde('haemoglobin')

kde('red_blood_cell_count')

In [ ]:
kde('white_blood_cell_count')

In [ ]:
kde('packed_cell_volume')

In [ ]:
kde('haemoglobin')

In [ ]:
kde('albumin')

In [ ]:
kde('blood_glucose_random')

In [ ]:
kde('specific_gravity')

In [ ]:
kde('sodium')

In [ ]:
kde('blood_urea')

In [ ]:
# Null-werte prüfen

df.isna().sum().sort_values(ascending = False)

In [ ]:
df[num_cols].isnull().sum()

In [ ]:
df[cat_cols].isnull().sum()

In [ ]:
# Null-Werte füllen mit random sampling für höhere Werte and mean/mode sampling für niedrigere Werte

def random_value_imputation(feature):
    random_sample = df[feature].dropna().sample(df[feature].isna().sum())
    random_sample.index = df[df[feature].isnull()].index
    df.loc[df[feature].isnull(), feature] = random_sample

def impute_mode(feature):
    mode = df[feature].mode()[0]
    df[feature] = df[feature].fillna(mode)

In [ ]:
df.isna().sum().sort_values(ascending = False)

In [ ]:
df[num_cols].isnull().sum()

In [ ]:
# Null-Werte "red_blood_cells" und "pus_cell" auffüllen mit random sampling Methode und cat_cols auffüllen mit mode imputation

random_value_imputation('red_blood_cells')
random_value_imputation('pus_cell')

for col in cat_cols:
    impute_mode(col)

In [ ]:
#Null-Werte prüfen
df[cat_cols].isnull().sum()

In [ ]:
#prüfen wieviele verschiedene Werte die kategorischen Spalten haben
for col in cat_cols:
    print(f"{col} has {df[col].nunique()} categories\n")

In [ ]:
#LabelEncoder um kategorische Werte in nummerische umzuwandeln

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in cat_cols:
    df[col] = le.fit_transform(df[col])

In [ ]:
df.head()

In [ ]:
#Aufteilung in unabhängige und abhängige Variablen
ind_col = [col for col in df.columns if col != 'class']
dep_col = 'class'

X = df[ind_col]
y = df[dep_col]

In [ ]:
#wähle speziell die unabhängigen Variablen aus
ind_col = []
dep_col = 'class'

for col in df.columns:
  if col == 'specific_gravity':
      ind_col.append(col)
  if col == 'haemoglobin':
      ind_col.append(col)
  if col == 'packed_cell_volume':
      ind_col.append(col)

X = df[ind_col]
y = df[dep_col]

In [ ]:
X.info()

In [ ]:
# Aufteilung in Trainings- und Testdaten

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 0)

print(y_train.isnull().sum())


0


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Definiere eine Pipeline mit Imputer und KNeighborsClassifier
pipeline = Pipeline([
    ('imputer', SimpleImputer()),  # Hier kannst du die Strategie für die Imputation festlegen
    ('classifier', KNeighborsClassifier())
])

# Trainiere das Modell mit der Pipeline
pipeline.fit(X_train, y_train)

# Bewertung des Modells
knn_acc = accuracy_score(y_test, pipeline.predict(X_test))
print(f"Test Accuracy of KNN is {knn_acc}")


## Anwendung des trainierten KNN in einer Applikation. Die Daten für 'specific_gravity', 'haemoglobin' und 'packed_cell_colume' eintragen und direkt eine Auswertung bekommen, ob Krankheit vorliegt

In [ ]:
# Eingabeaufforderungen für den Benutzer
specific_gravity = input("Geben Sie den Wert für 'specific_gravity' ein: ")
haemoglobin = input("Geben Sie den Wert für 'haemoglobin' ein: ")
packed_cell_volume = input("Geben Sie den Wert für 'packed_cell_volume' ein: ")

# Erstellung eines DataFrame aus den Benutzereingaben
# Wir müssen die Daten in ein "pd.Dataframe" Format bringen, weil knn.predict() solch ein Format erwartet.
data_to_predict = pd.DataFrame({
    'specific_gravity': [float(specific_gravity)],
    'haemoglobin': [float(haemoglobin)],
    'packed_cell_volume': [float(packed_cell_volume)]
})

# Verwendung des trainierten Modells zur Vorhersage
predictions = knn.predict(data_to_predict)

# Ausgabe der Vorhersage mit benutzerfreundlicher Nachricht
if predictions[0] == 0:
    print("Der Patient hat eine chronische Nierenerkrankung.")
else:
    print("Der Patient hat keine chronische Nierenerkrankung.")